# Match file

In [2]:
import pandas as pd
import numpy as np
import os
import re

In [3]:
def get_sec(row,col):
    s=row[col]
    l = s.split(':')
    return int(l[0]) * 3600 + int(l[1]) * 60 + int(l[2])

Mdf={}
Datapath='./../Matchjrn/'
Matchfilepath='./../biopacMatch/'
Biofilepath="./../biopac/"
biofilelist=[]
for fn in os.listdir(Biofilepath):
    biofilelist.append(fn)

num=0
for fn in os.listdir(Datapath):
    num=num+1
    sujet = re.split(r'\s*[.\-\s]\s*', fn)[1]  # M - S2.csv
    print num, sujet
    matchfile = Datapath + fn
    #print matchfile
    Mdfname=sujet
    df=pd.read_csv(matchfile,sep=',',header=None,index_col=0)
    df[3]=df.apply (lambda row: get_sec(row,1),axis=1)
    df[4]=df.apply (lambda row: get_sec(row,2),axis=1)
    df[5]=df[4]-df[3]
    df=df.loc[['base0','en1','en2','base1','m11','m12','base2','m21','m22','base3','m31','m32'],:]
    df.loc[['base0','base1','base2','base3'],6]=df.loc[:,4]-180
    df.loc[['en1','en2','m11','m12','m21','m22','m31','m32'],6]=df.loc[:,4]-240
    df[6]=df[6].astype(int)
    #df_o=df[[6,4]]
    Mdf[Mdfname]=df
    #df=pd.read_csv(matchfile,sep=',',header=None,index_col=0)
    for i in range(len(df)):
        mfile=Matchfilepath+sujet+'_'+str(df.index[i])+'.txt'
        #print mfile
        mbegin=df.ix[i,6]
        mend=df.ix[i,4]
        for j in biofilelist:
            if j.split('_')[0]==sujet:
                biopacfile=j
        cmd = "sed -n '/^" + str(mbegin) + "/,/^" + str(mend) + "/p' "+"./../biopac/"+ biopacfile + ">"+mfile
        #print cmd
        #os.system(cmd)

1 S18
sed -n '/^48947/,/^49127/p' ./../biopac/S18_2016-05-27T13_32_46.txt>./../biopacMatch/S18_base0.txt
sed -n '/^49198/,/^49438/p' ./../biopac/S18_2016-05-27T13_32_46.txt>./../biopacMatch/S18_en1.txt
sed -n '/^49573/,/^49813/p' ./../biopac/S18_2016-05-27T13_32_46.txt>./../biopacMatch/S18_en2.txt
sed -n '/^49970/,/^50150/p' ./../biopac/S18_2016-05-27T13_32_46.txt>./../biopacMatch/S18_base1.txt
sed -n '/^50250/,/^50490/p' ./../biopac/S18_2016-05-27T13_32_46.txt>./../biopacMatch/S18_m11.txt
sed -n '/^50645/,/^50885/p' ./../biopac/S18_2016-05-27T13_32_46.txt>./../biopacMatch/S18_m12.txt
sed -n '/^51947/,/^52127/p' ./../biopac/S18_2016-05-27T13_32_46.txt>./../biopacMatch/S18_base2.txt
sed -n '/^52203/,/^52443/p' ./../biopac/S18_2016-05-27T13_32_46.txt>./../biopacMatch/S18_m21.txt
sed -n '/^52593/,/^52833/p' ./../biopac/S18_2016-05-27T13_32_46.txt>./../biopacMatch/S18_m22.txt
sed -n '/^53889/,/^54069/p' ./../biopac/S18_2016-05-27T13_32_46.txt>./../biopacMatch/S18_base3.txt
sed -n '/^54177/

AttributeError: ("'float' object has no attribute 'split'", u'occurred at index a1')

# Evaluation

In [5]:
Datapath='./../../Data/'
evaluationfile=Datapath+'evaluationn.csv'
dfe=pd.read_csv(evaluationfile,sep=',')
#add sequence label(entrainement-0,match1-1,match2-2,match3-3)+(1°mitemps-1, 2°mitemps-2) 01,02
dfe.ix[dfe['Numéro du Match']=='entrainement',2]=0
dfe.ix[dfe['Quelle mi-temps ?']=='1° Mi-temps',3]=1
dfe.ix[dfe['Quelle mi-temps ?']=='2° Mi-temps',3]=2
dfe['seqlabel']=dfe.apply(lambda row: str(row['Numéro du Match'])+str(row['Quelle mi-temps ?']),axis=1)
#Add Subject label: S2-S58
dfe['sujet']=pd.Series(['S2']*6+map(lambda t: t[0]+str(t[1]), 
    zip(['S']*56*8,[i for i in range(3,59) for _ in range(8)])))

dfe.columns = ['horodateur', 'nom','NoMatch','Nomitemps',
               'noteg','q1','q2','q3','q4','q5','q6','q7','q8',
               'change','emotion','noteI','noteD','noteA','eventchange','quelevent','seqlabel','sujet']
dfe

,horodateur,nom,NoMatch,Nomitemps,noteg,q1,q2,q3,q4,q5,...,q8,change,emotion,noteI,noteD,noteA,eventchange,quelevent,seqlabel,sujet
0,11/05/2016 10:46:02,Rhouzlane Elias,1,1,5,Pas vraiment,Pas vraiment,Un peu,Un peu,Pas vraiment,...,Pas du tout,Peut-être,"Peur, Frustration",3,2,3,Oui,Hors-jeu,11,S2
1,11/05/2016 10:53:34,Rhouzlane Elias,1,2,6,Un peu,Pas vraiment,Plutôt,Plutôt,Un peu,...,Pas du tout,Oui,"Fierté, Frustration",3,2,3,Oui,Action décisive,12,S2
2,11/05/2016 11:20:04,Rhouzlane Elias,2,1,4,Pas vraiment,Pas vraiment,Plutôt,Pas vraiment,Un peu,...,Pas du tout,Non,"Peur, Frustration",3,3,2,Non,NaN,21,S2
3,11/05/2016 11:26:48,Rhouzlane Elias,2,2,4,Pas vraiment,Pas vraiment,Plutôt,Un peu,Plutôt,...,Pas du tout,Oui,"Peur, Colère, Frustration",4,3,3,Oui,But adverse,22,S2
4,11/05/2016 11:53:00,Rhouzlane Elias,3,1,6,Un peu,Pas vraiment,Un peu,Un peu,Pas du tout,...,Pas vraiment,Oui,"Joie, Fierté, Frustration",4,1,3,Oui,"But, lag",31,S2
5,11/05/2016 12:00:06,Rhouzlane Elias,3,2,6,Un peu,Un peu,Plutôt,Un peu,Pas du tout,...,Pas vraiment,Oui,"Joie, Fierté, Frustration",4,1,3,Oui,Tir raté,32,S2
6,12/05/2016 14:04:38,Cairoche Gabrielle,0,1,5,Pas du tout,Pas vraiment,Plutôt,Un peu,Un peu,...,Pas vraiment,Oui,"Fierté, Colère, Frustration",5,3,4,Non,NaN,01,S3
7,12/05/2016 14:05:54,Cairoche Gabrielle,0,2,4,Pas du tout,Pas du tout,Beaucoup,Pas vraiment,Beaucoup,...,Un peu,Oui,"Fierté, Colère, Frustration",5,5,2,Non,NaN,02,S3
8,12/05/2016 14:17:06,Cairoche Gabrielle,1,1,2,Pas du tout,Pas du tout,Beaucoup,Un peu,Plutôt,...,Un peu,Non,"Colère, Frustration",5,5,3,Oui,Le premier but,11,S3
9,12/05/2016 14:24:31,Cairoche Gabrielle,1,2,6,Un peu,Un peu,Beaucoup,Plutôt,Plutôt,...,Pas vraiment,Oui,"Joie, Fierté, Colère, Frustration",5,4,4,Non,NaN,12,S3


In [6]:
def evanote(option):
    return {
        'Pas du tout': 1,
        'Pas vraiment': 2,
        'Un peu':3,
        'Plutôt': 4,
        'Beaucoup': 5,
    }[option]

def qnote(row):
    amu=(evanote(row['q1'])+evanote(row['q4']))/2
    diffi= ((6-evanote(row['q2']))+evanote(row['q5']))/2
    impli=(evanote(row['q6'])+(6-evanote(row['q6'])))/2
    return (amu,diffi,impli)
#q1: joyeux
#q2: bon
#q3: frustré
#q4: amusant
#q5: difficile
#q6: concentré
#q7: autre chose
#q8: ennuyé

In [7]:
dfe['qnote']=dfe.apply (lambda row: qnote(row),axis=1)
dfe['qI']=dfe.apply(lambda row: row['qnote'][2],axis=1)
dfe['qD']=dfe.apply(lambda row: row['qnote'][1],axis=1)
dfe['qA']=dfe.apply(lambda row: row['qnote'][0],axis=1)
dfe.ix[dfe['eventchange']=='Oui','eventchange']=1
dfe.ix[dfe['eventchange']=='Non','eventchange']=0
dfe.ix[dfe['change']=='Oui','change']=1
dfe.ix[dfe['change']=='Non','change']=0
dfe.ix[dfe['change']=='Peut-être','change']=-1
evadf=dfe[['sujet','seqlabel','NoMatch','Nomitemps','noteg','change','eventchange','noteI','noteD','noteA','qI','qD','qA']]
evadf

,sujet,seqlabel,NoMatch,Nomitemps,noteg,change,eventchange,noteI,noteD,noteA,qI,qD,qA
0,S2,11,1,1,5,-1,1,3,2,3,3,3,2
1,S2,12,1,2,6,1,1,3,2,3,3,3,3
2,S2,21,2,1,4,0,0,3,3,2,3,3,2
3,S2,22,2,2,4,1,1,4,3,3,3,4,2
4,S2,31,3,1,6,1,1,4,1,3,3,2,3
5,S2,32,3,2,6,1,1,4,1,3,3,2,3
6,S3,01,0,1,5,1,0,5,3,4,3,3,2
7,S3,02,0,2,4,1,0,5,5,2,3,5,1
8,S3,11,1,1,2,0,1,5,5,3,3,4,2
9,S3,12,1,2,6,1,0,5,4,4,3,3,3


# Evaluation global

In [8]:
Datapath='./../../Data/'
evaluationfile=Datapath+'evaglobal.csv'
dfg1=pd.read_csv(evaluationfile,sep=',')
l=len(dfg1)
dfg1['sujet']=pd.Series(
    map(lambda t: t[0]+str(t[1]),
    zip(['S']*l,range(1,l+1))
       )
    )
dfg1['D']=dfg1['Difficulté'].map(lambda x: x.split('>'))
dfg1['I']=dfg1['Implication'].map(lambda x: x.split('>'))
dfg1['A']=dfg1['Amusement'].map(lambda x: x.split('>'))
dfg1



,Horodateur,Nom/prénom,Difficulté,Implication,Amusement,sujet,D,I,A
0,09/05/2016 14:38:18,Cyrille KAMDEM,1>0>2>3,3>0>2>1,3>0>2>1,S1,"[1, 0, 2, 3]","[3, 0, 2, 1]","[3, 0, 2, 1]"
1,11/05/2016 12:02:11,Rhouzlane Elias,1>0>2>3,2>0>1>3,2>0>3>1,S2,"[1, 0, 2, 3]","[2, 0, 1, 3]","[2, 0, 3, 1]"
2,12/05/2016 15:30:33,Cairoche Gabrielle,1>0>2>3,1>0>3>2,3>1>0>2,S3,"[1, 0, 2, 3]","[1, 0, 3, 2]","[3, 1, 0, 2]"
3,12/05/2016 18:18:19,GOMEZ DAVID,3>1>0>2,3>2>1>0,3>2>1>0,S4,"[3, 1, 0, 2]","[3, 2, 1, 0]","[3, 2, 1, 0]"
4,13/05/2016 12:10:13,RABIA,1>0>3>2,2>3>1>0,3>2>1>0,S5,"[1, 0, 3, 2]","[2, 3, 1, 0]","[3, 2, 1, 0]"
5,13/05/2016 15:34:20,Démétrius/David,1>2>3>0,0>1>2>3,3>2>1>0,S6,"[1, 2, 3, 0]","[0, 1, 2, 3]","[3, 2, 1, 0]"
6,13/05/2016 18:07:01,LE Dao Khanh Vu,2>3>1>0,1>0>3>2,1>0>3>2,S7,"[2, 3, 1, 0]","[1, 0, 3, 2]","[1, 0, 3, 2]"
7,20/05/2016 12:02:43,Dany Naomi,2>0>1>3,0>1>2>3,1>3>2>0,S8,"[2, 0, 1, 3]","[0, 1, 2, 3]","[1, 3, 2, 0]"
8,20/05/2016 15:10:28,kama / babacar,1>2>3>0,1>2>3>0,3>1>2>0,S9,"[1, 2, 3, 0]","[1, 2, 3, 0]","[3, 1, 2, 0]"
9,20/05/2016 17:55:38,Fontaine emile,0>1>2>3,1>2>3>0,2>0>1>3,S10,"[0, 1, 2, 3]","[1, 2, 3, 0]","[2, 0, 1, 3]"


In [9]:
Datapath='./../../Data/'
evaluationfile=Datapath+'evaglobal1.csv'
dfg2=pd.read_csv(evaluationfile,sep=',')
l2=len(dfg2)
dfg2['sujet']=pd.Series(
    map(lambda t:t[0]+str(t[1]),zip(['S']*l2,range(18,18+l2)))
)
tmp_diffi=dfg2[['sujet','D0','D1','D2','D3']]
s_diffi=pd.melt(tmp_diffi,id_vars=['sujet']).pivot(index='sujet', columns='value', values='variable').reset_index()
dfg2['D']=s_diffi.apply(lambda row: [int(row[1].split('D')[1]),int(row[2].split('D')[1]),int(row[3].split('D')[1]),int(row[4].split('D')[1])]
              ,axis=1)

tmp_impli=dfg2[['sujet','I0','I1','I2','I3']]
s_impli=pd.melt(tmp_impli,id_vars=['sujet']).pivot(index='sujet', columns='value', values='variable').reset_index()
dfg2['I']=s_impli.apply(lambda row: [int(row[1].split('I')[1]),int(row[2].split('I')[1]),int(row[3].split('I')[1]),int(row[4].split('I')[1])]
              ,axis=1)

tmp_amu=dfg2[['sujet','A0','A1','A2','A3']]
s_amu=pd.melt(tmp_amu,id_vars=['sujet']).pivot(index='sujet', columns='value', values='variable').reset_index()
dfg2['A']=s_amu.apply(lambda row: [int(row[1].split('A')[1]),int(row[2].split('A')[1]),int(row[3].split('A')[1]),int(row[4].split('A')[1])]
              ,axis=1)
tmp_exci=dfg2[['sujet','E0','E1','E2','E3']]
s_exci=pd.melt(tmp_exci,id_vars=['sujet']).pivot(index='sujet', columns='value', values='variable').reset_index()
dfg2['E']=s_exci.apply(lambda row: [int(row[1].split('E')[1]),int(row[2].split('E')[1]),int(row[3].split('E')[1]),int(row[4].split('E')[1])]
              ,axis=1)

dfg=dfg1[['sujet','D','I','A']].append(dfg2[['sujet','D','I','A']],ignore_index=True)
dfg

In [20]:
evadf[(evadf['Nomitemps']==2) & (evadf['sujet']!='S2')]

,sujet,seqlabel,NoMatch,Nomitemps,noteg,change,eventchange,noteI,noteD,noteA,qI,qD,qA
7,S3,02,0,2,4,1,0,5,5,2,3,5,1
9,S3,12,1,2,6,1,0,5,4,4,3,3,3
11,S3,22,2,2,4,1,1,4,4,2,3,3,2
13,S3,32,3,2,7,1,1,4,3,3,3,2,4
15,S4,02,0,2,7,-1,1,3,3,4,3,2,3
17,S4,12,1,2,8,1,1,4,5,5,3,4,4
19,S4,22,2,2,8,-1,1,5,2,4,3,1,4
21,S4,32,3,2,8,1,1,5,4,5,3,3,4
23,S5,02,0,2,7,-1,0,4,3,4,3,3,3
25,S5,12,1,2,8,1,1,5,4,4,3,4,4


In [11]:
def rank1(metric):
    dfg['1']=dfg.apply(lambda row: row[metric][0],axis=1)
    dfg['2']=dfg.apply(lambda row: row[metric][1],axis=1)
    dfg['3']=dfg.apply(lambda row: row[metric][2],axis=1)
    dfg['4']=dfg.apply(lambda row: row[metric][3],axis=1)
    return dfg[['sujet','1','2','3','4']]

In [105]:
implicationrank=rank1('I')
amusantrank=rank1('A')
difficilerank=rank=rank1('D')

In [106]:
implicationrank

,sujet,1,2,3,4
0,S1,3,0,2,1
1,S2,2,0,1,3
2,S3,1,0,3,2
3,S4,3,2,1,0
4,S5,2,3,1,0
5,S6,0,1,2,3
6,S7,1,0,3,2
7,S8,0,1,2,3
8,S9,1,2,3,0
9,S10,1,2,3,0


In [96]:
dfg2

,Horodateur,Nom/prénom,I0,I1,I2,I3,D0,D1,D2,D3,...,A3,E0,E1,E2,E3,sujet,D,I,A,E
0,27/05/2016 15:16:56,Azizi Yacine,1,3,2,4,3,4,1,2,...,4,2,3,1,4,S18,"[2, 3, 0, 1]","[0, 2, 1, 3]","[0, 2, 1, 3]","[2, 0, 1, 3]"
1,27/05/2016 18:11:31,aznavour alexandre,1,2,3,4,4,3,1,2,...,2,4,3,1,2,S19,"[2, 3, 1, 0]","[0, 1, 2, 3]","[1, 3, 0, 2]","[2, 3, 1, 0]"
2,30/05/2016 18:25:14,Kissié Christopher,2,1,4,3,4,3,1,2,...,3,3,2,4,1,S20,"[2, 3, 1, 0]","[1, 0, 3, 2]","[1, 0, 3, 2]","[3, 1, 0, 2]"
3,31/05/2016 15:27:37,Staiano Jacopo,3,2,1,4,2,3,1,4,...,3,4,1,3,2,S21,"[2, 0, 1, 3]","[2, 1, 0, 3]","[1, 2, 3, 0]","[1, 3, 2, 0]"
4,31/05/2016 17:55:01,SELLE-DELAVAUD Lorraine,2,1,3,4,2,1,3,4,...,4,2,1,3,4,S22,"[1, 0, 2, 3]","[1, 0, 2, 3]","[1, 0, 2, 3]","[1, 0, 2, 3]"
5,01/06/2016 11:52:33,Khelifi Lilia,4,3,1,2,1,2,4,3,...,2,4,3,1,2,S23,"[0, 1, 3, 2]","[2, 3, 1, 0]","[2, 3, 0, 1]","[2, 3, 1, 0]"
6,02/06/2016 11:45:53,Antunes Da Silva Victor,3,1,4,2,3,1,4,2,...,4,4,3,1,2,S24,"[1, 3, 0, 2]","[1, 3, 0, 2]","[2, 0, 1, 3]","[2, 3, 1, 0]"
7,02/06/2016 15:30:38,KIMBIDIMA Kevin,4,3,2,1,4,3,2,1,...,4,4,2,1,3,S25,"[3, 2, 1, 0]","[3, 2, 1, 0]","[1, 0, 2, 3]","[2, 1, 3, 0]"
8,02/06/2016 18:28:22,galiano nicolas,2,1,4,3,3,1,4,2,...,3,3,2,4,1,S26,"[1, 3, 0, 2]","[1, 0, 3, 2]","[1, 0, 3, 2]","[3, 1, 0, 2]"
9,03/06/2016 12:07:57,Porlon Janice,4,3,2,1,2,4,3,1,...,3,4,3,1,2,S27,"[3, 0, 2, 1]","[3, 2, 1, 0]","[2, 1, 3, 0]","[2, 3, 1, 0]"


In [98]:
dfg2_impli=pd.read_csv(evaluationfile,sep=',',usecols=range(0,6))
dfg2_diffi=pd.read_csv(evaluationfile,sep=',',usecols=[0,1,6,7,8,9])
dfg2_impli
tmp=pd.melt(dfg2,id_vars=['Horodateur','Nom/prénom']).reset_index()
tmp


,index,Horodateur,Nom/prénom,variable,value
0,0,27/05/2016 15:16:56,Azizi Yacine,I0,1
1,1,27/05/2016 18:11:31,aznavour alexandre,I0,1
2,2,30/05/2016 18:25:14,Kissié Christopher,I0,2
3,3,31/05/2016 15:27:37,Staiano Jacopo,I0,3
4,4,31/05/2016 17:55:01,SELLE-DELAVAUD Lorraine,I0,2
5,5,01/06/2016 11:52:33,Khelifi Lilia,I0,4
6,6,02/06/2016 11:45:53,Antunes Da Silva Victor,I0,3
7,7,02/06/2016 15:30:38,KIMBIDIMA Kevin,I0,4
8,8,02/06/2016 18:28:22,galiano nicolas,I0,2
9,9,03/06/2016 12:07:57,Porlon Janice,I0,4


# annotation

In [12]:
import os
for fn in os.listdir('./../Data/annotation/'):
    l=fn.split(' - ')
    t=l[1].split('_')
    sujet=t[0]
    matnum=t[1][-1]
    matname=t[2]    
    niveau={
        'debutant':1,
        'amateur':2,
        'semipro':3,
        'pro':4,
        'champion':5,
        'legend':6
    }[matname]
   

3
1
2
2
2
2
2
1
1
3
1
2
2
1
1
3
3
3
3
2
1
3
3
3
2
1
3
1
2
1


In [11]:
k='12d'
k[-1]

'd'